In [1]:
import pickle

for year in [2020,2021,2022,2023]:
    results_dict = {}
    with open("sims_stackexchange_results_" + str(year) + ".pkl", "rb") as f:
        results_dict = pickle.load(f)
    print(results_dict.keys())

dict_keys([0, 1, 2, 3])
dict_keys([0, 1, 2, 3, 4])
dict_keys([0, 1, 2, 3, 4])
dict_keys([0, 1, 2, 3, 4])


In [23]:
results_dict[0][10][1]['final_centroids'].shape

(35, 1536)

In [ ]:
import numpy as np
results_summary = {}
for year in [2020,2021,2022,2023]:
    results_summary[year] = {}
    results_dict = {}
    with open("sims_stackexchange_results_" + str(year) + ".pkl", "rb") as f:
        results_dict = pickle.load(f)
    
    for method in ['minibatchkmeans','seqminibatchkmeans','kmeans']:   
        results_summary[year][method] = {}
        acc = []
        nmi = []
        cen = []
        
        for i in results_dict.keys():
    
            acc.append(results_dict[i][method]['results'][0])
            nmi.append(results_dict[i][method]['results'][1])
            cen.append(results_dict[i][method]['results'][2])
            
        results_summary[year][method] = list(np.round([100*np.mean(acc), 100*np.mean(nmi)],1)) + [np.round(np.mean(cen),3)]
        
    for force in [10, 50]:
            for niter in [1,5]:
                acc = []
                nmi = []
                cen = []
                
                for i in results_dict.keys():
                    try:
                        cur_results = results_dict[i][force][niter]['results']
                        acc.append(cur_results[0])
                        nmi.append(cur_results[1])
                        cen.append(cur_results[2])
                    except:
                        continue
                results_summary[year][str(force) + '-' + str(niter)] = list(np.round([100*np.mean(acc), 100*np.mean(nmi)],1)) + [np.round(np.mean(cen),3)]

In [ ]:
line = {}
    
for year in [2020,2021,2022,2023]:

    acc = []
    nmi = []
    cen = []
    method_names = []
    for method in ['kmeans', 'minibatchkmeans','seqminibatchkmeans','50-1','50-5','10-1','10-5',]:
        cur_results = results_summary[year][method]
        acc.append(cur_results[0])
        nmi.append(cur_results[1])
        cen.append(cur_results[2])
        if method == '10-1':
            cur_method = 'mini-batch k-LLMmeans-FS1'
        elif method == '10-5':
            cur_method = 'mini-batch k-LLMmeans-FS5'
        elif method == '50-1':
            cur_method = 'mini-batch k-LLMmeans-1'
        elif method == '50-5':
            cur_method = 'mb k-LLMmeans-5'
        elif method == 'kmeans':
            cur_method = 'k-means'
        elif method == 'minibatchkmeans':
            cur_method = 'mini-batch k-means'
        elif method == 'seqminibatchkmeans':
            cur_method = 'seq. mini-batch k-means'
        method_names.append(cur_method)

    # Find the maximum value
    val = max(acc)
    acc = [f"{{\\bf {x}}}" if x == val else x for x in acc]

    val = max(nmi)
    nmi = [f"{{\\bf {x}}}" if x == val else x for x in nmi]

    val = min(cen)
    cen = [f"{{\\bf {x}}}" if x == val else x for x in cen]
    
    for ii in range(len(method_names)):
        cur_method = method_names[ii]
        if year == 2020:
            if cur_method == 'k-means':
                line[cur_method] = cur_method + '&' + str(nmi[ii]) + '&' + str(cen[ii])
            else:
                line[cur_method] = cur_method + '&' + str(nmi[ii]) + '&' + str(cen[ii])
        else:
            line[cur_method] = line[cur_method] + '&&' + str(nmi[ii]) + '&' + str(cen[ii])

text = ""
for ii in range(len(method_names)):
    cur_method = method_names[ii]
    text = text + line[cur_method] + '\\\\\n'
print(text)          